# ETL dataset <code>users_items.gz.json</code>

En este notebook, nuestro objetivo es realizar el proceso de extracción, transformación y carga (ETL) de los datos que contienen información sobre reseñas realizadas por los usuarios de Steam. Así como preparar los datos para su posterior análisis y modelado.

## Configuraciones Globales e Importaciones

En esta sección, importamos todas las bibliotecas y/o módulos necesarios para nuestro proceso ETL 

In [1]:
import sys
import os
from builtins import print

import pandas as pd
import gzip
import ast

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

System version: 3.11.8 (main, Feb 13 2024, 10:08:31) [GCC 12.2.0]
Pandas version: 2.2.1


## Extracción

En esta sección, extraemos los datos del archivo `users_items.gz.json` y describimos a detalle su contenido.

### Extracción de los datos

In [2]:
# Ruta al dataset
path = '/app/dataset/users_items.json.gz'

# Leemos el archivo usando ast.literal_eval para analizar la cadena JSON
data = []
with gzip.open(path, 'r') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode("utf-8")))

# Convertimos a DataFrame
df_items = pd.DataFrame(data)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


- Hacemos un resumen conciso del Dataframe para observar los tipos de datos por columnas, verificar nulos.

In [3]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


- Observamos que la estructura de los datos de la columna 'items' parece contener una lista de diccionarios.

In [4]:
df_items['items'][1][:3]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '80',
  'item_name': 'Counter-Strike: Condition Zero',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '100',
  'item_name': 'Counter-Strike: Condition Zero Deleted Scenes',
  'playtime_forever': 0,
  'playtime_2weeks': 0}]

- Comprobamos que efectivamente es una lista de diccionarios y procedemos a desanidarlo.

In [4]:
meta = df_items.columns[:4].tolist()
df_items_expanded = pd.json_normalize(data, 'items', meta)
df_items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


* Comprobamos los datos del DataFrame desanidado.

In [5]:
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       object
 6   steam_id          object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


### Descripción de los datos.

A partir del análisis exploratorio anterior, podemos observar que el conjunto de datos contiene **5153209** registros y **8** variables con información de los usuarios de Steam y el contenido que consumen. A continuación, se describe el contenido de las variables:

- **user_id**: identificador único del usuario.
- **items_count**: número total de juegos que posee el usuario
- **steam_id**: identificador único SteamID de la cuenta.
- **user_url**: URL del perfil del usuario.
- **items**: items del usuario en formato JSON. Contiene las siguientes claves:
    - **item_id**: identificador único del juego.
    - **item_name**: nombre del juego.
    - **playtime_forever**:  número total de minutos jugados desde que se tiene registro (año 2009).
    - **playtime_2weeks**: número total de minutos jugados en las últimas 2 semanas.



## Transformación

En esta sección, realizamos la limpieza inicial de los datos y las transformaciones necesarias. Esto puede incluir la creación de nuevas columnas a partir de las existentes, la eliminación de duplicados o columnas innecesarias, la gestión de valores nulos o la corrección de tipos de datos.

### Gestión de valores nulos

Comprobamos nulos:

In [6]:
df_items_expanded.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

No tenemos nulos por lo que ahora comprobamos duplicados.

### Verificación de duplicados

Comprobamos si tenemos duplicados:

In [7]:
df_items_expanded[df_items_expanded.duplicated(keep=False)].sort_values(['user_id','item_id'])

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
1094154,104900,ORION: Prelude,0,0,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129
2712151,104900,ORION: Prelude,0,0,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129
1094152,105600,Terraria,650,0,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129
2712149,105600,Terraria,650,0,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129
1094156,17080,Tribes: Ascend,0,0,05041129,35,76561198167088451,http://steamcommunity.com/id/05041129
...,...,...,...,...,...,...,...,...
2928853,96400,Shoot Many Robots,759,0,zeroblade,306,76561197970272666,http://steamcommunity.com/id/zeroblade
1226806,97100,Section 8: Prejudice,70,0,zeroblade,306,76561197970272666,http://steamcommunity.com/id/zeroblade
2928821,97100,Section 8: Prejudice,70,0,zeroblade,306,76561197970272666,http://steamcommunity.com/id/zeroblade
1226872,98200,Frozen Synapse,112,0,zeroblade,306,76561197970272666,http://steamcommunity.com/id/zeroblade


In [8]:
df_items_expanded.duplicated().sum()

59104

Tenemos **59104** duplicados que debemos a eliminar.

In [9]:
df_items_expanded.drop_duplicates(inplace=True)
df_items_expanded.shape

(5094105, 8)

### Conversión de minutos a horas en `playtime_forever`

Como queremos que nuestra API nos devuelva el tiempo de juego en horas y no en minutos, convertimos la columna `playtime_forever` a la unidad adecuada. 

In [10]:
df_items_expanded['playtime_forever'] = df_items_expanded['playtime_forever'] / 60
df_items_expanded.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,0.100000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0.000000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,0.116667,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0.000000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0.000000,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


### Eliminación de columnas innecesarias

Para optimizar nuestras consultas en la API, eliminamos las columnas `playtime_2weeks`, `steam_id`, `user_url`, `item_name` y `items_count`, que tampoco son necesarias para nuestro análisis. 



In [11]:
columns = ['playtime_2weeks', 'steam_id', 'user_url', 'item_name','items_count']
df_items_expanded.drop(columns, axis=1, inplace=True)
df_items_expanded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   playtime_forever  float64
 2   user_id           object 
dtypes: float64(1), object(2)
memory usage: 155.5+ MB


## Carga

##### Cargamos el dataset con el dataset limpio y formato comprimido.

In [12]:
# Definimos la ruta de destino y el nombre del archivo.
folder_path = '/app/dataset/'
file_name = 'user_items.parquet'

# Verificamos si el folder_path existe
if not os.path.exists(folder_path):
    # Si no existe, lo creamos
    os.makedirs(folder_path)

# Exportamos el DataFrame a un archivo Parquet
df_items_expanded.to_parquet(os.path.join(folder_path, file_name), engine='pyarrow', compression='zstd')

print(f'El archivo {file_name} se guardó correctamente en {folder_path}')

El archivo user_items.parquet se guardó correctamente en /app/dataset/
